$\textbf{Ejercicio}$

1. Entrena y calibra un árbol de decisión para la base de datos moons

$\bullet$ Genera una base de datos usando make_moons(n_samples=10_000, noise=0.4, random_state=25) (la función se importa desde sklearn.datasets)  

$\bullet$ Divide entre train y test set usando train_test_split  

$\bullet$ Realiza una búsqueda con cross-validation para encontrar hiperparámetros que mejoren un modelo de DecisionTreeClassifier. Comenta tus resultados y lo que encontró el modelo

In [2]:
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from tqdm.notebook import tqdm
from time import sleep
from sklearn.datasets import make_moons
from sklearn.tree import DecisionTreeClassifier, export_graphviz, plot_tree
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_validate,train_test_split

In [3]:
#Genera una base de datos usando make_moons(n_samples=10_000, noise=0.4, random_state=25) (la función se importa desde sklearn.datasets)
moons = make_moons(n_samples=10_000, noise=0.4, random_state=25)
moons

(array([[-0.09821478,  0.39829029],
        [-0.37972495,  0.53213178],
        [ 1.3341397 ,  0.23372859],
        ...,
        [-0.3256327 ,  0.65045201],
        [ 0.27601752, -0.29412182],
        [ 1.09774777, -0.37445723]]),
 array([0, 0, 1, ..., 0, 1, 1], dtype=int64))

In [4]:
#Divide entre train y test set usando train_test_split 
X,y=moons
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
#Encontramos los mejores hiperparámetros basados en un rango de nodos y muestras
#Por defecto, GridSearchCV entrena el mejor modelo para todo el conjunto de entrenamiento
from sklearn.model_selection import GridSearchCV

params = {'max_leaf_nodes': list(range(2, 200)), 'min_samples_split': [2, 3, 4, 5]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)

grid_search_cv.fit(X_train, y_train)


Fitting 3 folds for each of 792 candidates, totalling 2376 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 2376 out of 2376 | elapsed:    5.3s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
  

In [9]:
#Entrenamos el modelo basado en los mejores hiperparámetros
grid_search_cv.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=15,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [7]:
#El mejor modelo de alcanza tiene como maximos una descendencia de 15 nodos y mínimo una
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.849